In [5]:
import torch.nn as nn
import numpy as np
import torch

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [7]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [8]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [9]:
from torch.utils.data import TensorDataset

In [10]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [11]:
from torch.utils.data import DataLoader

In [12]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [13]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  96.,  71.],
        [ 73.,  67.,  43.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [102.,  43.,  37.]])
tensor([[104., 118.],
        [ 56.,  70.],
        [118., 134.],
        [ 20.,  38.],
        [ 22.,  37.]])


In [14]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1588, -0.0566,  0.3976],
        [-0.1725, -0.1435, -0.1389]], requires_grad=True)
Parameter containing:
tensor([ 0.0079, -0.2005], requires_grad=True)


In [15]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1588, -0.0566,  0.3976],
         [-0.1725, -0.1435, -0.1389]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0079, -0.2005], requires_grad=True)]

In [16]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 24.9059, -28.3812],
        [ 34.9255, -37.4171],
        [ 29.3015, -42.4960],
        [ 28.4841, -29.1050],
        [ 33.3646, -35.6043],
        [ 25.1213, -28.4102],
        [ 35.3797, -37.4125],
        [ 29.8579, -42.8074],
        [ 28.2687, -29.0761],
        [ 33.6033, -35.5707],
        [ 25.3601, -28.3766],
        [ 35.1409, -37.4461],
        [ 28.8473, -42.5006],
        [ 28.2453, -29.1386],
        [ 33.1492, -35.5753]], grad_fn=<AddmmBackward0>)

In [17]:
# Import nn.functional
import torch.nn.functional as F

In [18]:
# Define loss function
loss_fn = F.mse_loss

In [19]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10382.5654, grad_fn=<MseLossBackward0>)


In [20]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [21]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [23]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 16.0938
Epoch [20/1000], Loss: 13.1652
Epoch [30/1000], Loss: 8.7576
Epoch [40/1000], Loss: 13.0108
Epoch [50/1000], Loss: 10.2177
Epoch [60/1000], Loss: 10.4359
Epoch [70/1000], Loss: 12.5355
Epoch [80/1000], Loss: 6.0228
Epoch [90/1000], Loss: 11.5936
Epoch [100/1000], Loss: 5.4597
Epoch [110/1000], Loss: 9.7061
Epoch [120/1000], Loss: 4.0026
Epoch [130/1000], Loss: 5.2055
Epoch [140/1000], Loss: 8.8435
Epoch [150/1000], Loss: 5.7698
Epoch [160/1000], Loss: 3.4440
Epoch [170/1000], Loss: 2.0622
Epoch [180/1000], Loss: 4.5339
Epoch [190/1000], Loss: 5.2077
Epoch [200/1000], Loss: 3.6898
Epoch [210/1000], Loss: 3.1237
Epoch [220/1000], Loss: 2.6638
Epoch [230/1000], Loss: 2.3849
Epoch [240/1000], Loss: 2.5804
Epoch [250/1000], Loss: 3.4439
Epoch [260/1000], Loss: 3.9200
Epoch [270/1000], Loss: 2.5620
Epoch [280/1000], Loss: 4.3948
Epoch [290/1000], Loss: 2.9881
Epoch [300/1000], Loss: 1.8276
Epoch [310/1000], Loss: 1.2157
Epoch [320/1000], Loss: 3.3610
Epoch [330

In [24]:
preds = model(inputs)
preds

tensor([[ 56.9177,  70.3921],
        [ 81.9731, 100.4202],
        [118.1603, 133.1416],
        [ 20.7920,  37.8578],
        [101.7896, 118.1907],
        [ 55.6710,  69.3121],
        [ 81.8279, 100.4890],
        [118.4532, 133.7312],
        [ 22.0388,  38.9378],
        [102.8911, 119.3396],
        [ 56.7724,  70.4610],
        [ 80.7264,  99.3401],
        [118.3056, 133.0728],
        [ 19.6906,  36.7089],
        [103.0364, 119.2708]], grad_fn=<AddmmBackward0>)

In [25]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [26]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4365, 67.5005]], grad_fn=<AddmmBackward0>)